### Conversational AI (Chatbot)

In [43]:
import os
from dotenv import load_dotenv
import gradio as gr
import google.generativeai as genai
from IPython.display import Markdown, display, clear_output

In [44]:
load_dotenv()

google_api_key = os.getenv('GEMINI_API_KEY')

In [45]:
genai.configure(api_key=google_api_key)

In [4]:
system_message = "You are a helpful assistant"

In [5]:
def message_gemini(prompt):
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",  # or "gemini-1.5-pro"
        system_instruction=system_message
    )
    
    response = model.generate_content(prompt)
    return response.text


In [6]:
view = gr.Interface(
    fn=message_gemini,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


### Streaming Output

In [ ]:

system_message = "You are a helpful assistant that responds in markdown"  # Optional system instruction

def stream_gemini(prompt):
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",
        system_instruction=system_message
    )

    stream = model.generate_content(prompt, stream=True)

    result = ""
    for chunk in stream:
        if chunk.text:
            result += chunk.text
            yield result


In [8]:
for partial in stream_gemini("Explain quantum computing in simple terms."):
    clear_output(wait=True)
    display(Markdown(partial))

## Quantum Computing Explained Simply

Regular computers store information as bits, which are like switches that can be either on (1) or off (0).  Quantum computers use **qubits**.  Qubits are much more versatile. Because of the weirdness of quantum mechanics, they can be:

* **0 or 1:** Just like a regular bit.
* **Both 0 and 1 at the same time:** This is called **superposition**. Imagine a coin spinning in the air – it's neither heads nor tails until it lands.
* **Entangled:** Multiple qubits can be linked together in a way that their fates are intertwined.  Measuring one instantly tells you the state of the others, no matter how far apart they are!


This allows quantum computers to explore many possibilities simultaneously.  Think of searching a maze: a regular computer tries each path one by one. A quantum computer can explore *all* paths at once, finding the exit much faster.

**Here's the analogy:**

Imagine you're searching for a specific grain of sand on a beach.

* **Classical computer:** Checks each grain of sand one by one.
* **Quantum computer:** Checks all grains of sand simultaneously.

**However, it's important to note:**

* Quantum computers aren't meant to replace regular computers. They're good at specific tasks, like drug discovery, materials science, and breaking certain types of encryption.
* They're very difficult and expensive to build and maintain.  They require extremely low temperatures and are highly sensitive to noise.
* They're still in their early stages of development.


In short, quantum computing leverages the bizarre rules of quantum mechanics to perform calculations in a fundamentally different and potentially much faster way than classical computers for specific problems.


In [9]:
view = gr.Interface(
    fn=stream_gemini,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=gr.Markdown(),
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


### Chat Bot With History

In [ ]:
def stream_gemini_with_history(message, history):
    
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",
        system_instruction=system_message
    )
    
    prompt = ""
    for user_msg, bot_msg in history:
        prompt += f"User: {user_msg}\nAssistant: {bot_msg}\n"
    prompt += f"User: {message}\nAssistant:"

    stream = model.generate_content(prompt, stream=True)

    response = ""
    for chunk in stream:
        if chunk.text:
            response += chunk.text
            yield history + [(message, response)], ""
